In [4]:
import geopandas as gpd
import requests
import json
from datetime import datetime

In [5]:
# Load the shapefile
shapefile_path = 'c:/temp/planet/waterbody.shp'
gdf = gpd.read_file(shapefile_path)
shapefile_wgs84  = gdf.to_crs("epsg:4269")

# Get the bounding box coordinates
bounding_box = shapefile_wgs84.bounds.values[0]
print(bounding_box)

[151.2994286  -27.75282169 151.31083805 -27.7450019 ]


In [8]:
# Define date range
start_date = '2022-12-20T00:00:00Z'  # Start date in ISO 8601 format
end_date = '2023-12-19T23:59:59Z'  # End date in ISO 8601 format

# Define other parameters such as item type (e.g., PSOrthoTile) and asset type (e.g., visual)
item_type = 'PSScene'  # Modify based on your requirement
asset_type = 'ortho_analytic_8b_sr'

# Define filters
filters = {
    "type": "AndFilter",
    "config": [
        {
            "type": "GeometryFilter",
            "field_name": "geometry",
            "config": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [bounding_box[0], bounding_box[1]],
                        [bounding_box[0], bounding_box[3]],
                        [bounding_box[2], bounding_box[3]],
                        [bounding_box[2], bounding_box[1]],
                        [bounding_box[0], bounding_box[1]]
                    ]
                ]
        },
        {
            "type": "DateRangeFilter",
            "field_name": "acquired",
            "config": {
                "gte": start_date,
                "lte": end_date
            }
        }
}
    ]
# Define request payload
request_payload = {
    "item_types": [item_type],
    "filter": filters
}


SyntaxError: invalid syntax (<ipython-input-8-5b2f6b211cd4>, line 36)

In [ ]:

# Define your Planet API key
API_KEY = '98c3d498f425463ab3b3d3634bcef6b2'

# Set authorization headers
headers = {
    'Authorization': f'api-key {API_KEY}',
    'Content-Type': 'application/json'
}

# Send request to Planet API
response = requests.post('https://api.planet.com/data/v1/quick-search', data=json.dumps(request_payload), headers=headers)

# Check if request was successful
if response.status_code == 200:
    # Parse the response
    results = response.json()['features']
    for result in results:
        # Get asset links
        assets = result['_links']['assets']
        # Download the asset
        asset_response = requests.get(assets[asset_type], headers=headers)
        # Save the asset to file
        with open(f"{result['id']}.tif", 'wb') as f:
            f.write(asset_response.content)
    print("Download complete.")
else:
    print("Error:", response.status_code)
    print(response.text)